In [1]:
import math
import sys
import random
import numpy as np
import re

alphs = ["abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ",
         "абвгдеёжзийклмнопрстуфхцчшщъыьэюя",
        "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz_", "abcdefghijklmnopqrstuvwxyz"]
map_alph={alphs[2][i] : i for i in range(0, len(alphs[2]))}

#НОД
def gcd(a, b):
    if b == 0: return a
    else: return gcd(b, a%b)
#is Simple
def isSimple(a):
    if a == 1: return False
    if a == 2: return True
    sq_a = math.ceil(math.sqrt(a))
    for i in range(2, sq_a+1):
        if a%i == 0:
            return False
    return True
#Получение списка простых чисел
def get_simples(n):
    v = []
    for i in range(2, n):
        if isSimple(i):
            v.append(i)
    return v
#функция Эйлера
def F(n):
    res = n
    for a in SIMPLES:
        if a >= math.ceil(math.sqrt(n))+1: break
        if n%a == 0:
            res -= res/a
            while n%a == 0:
                n /= a
    if n > 1:
        res -= res/n
    return res
#бинарное возведение в степень по модулю
def bin_pow(a, n, m):
    if n == 0: return 1
    if n == 1: return a
    if n%2 == 0: return bin_pow(a, n/2, m)%m*bin_pow(a, n/2, m)%m
    return bin_pow(a, (n//2)+1, m)%m*bin_pow(a, n//2, m)%m

In [3]:
#Caesar (Цезаря)
def encrypt(s, step, alph, map_alph):
    return "".join([alph[((map_alph[a]-step)%len(alph))] for a in s])
def decrypt(s, step):
    alph = alphs[1]
    return "".join([alph[((map_alph[a]+step)%len(alph))] for a in s])

In [86]:
#Caesar (UNICODE alph)
def decrypt(s, step):
    return "".join([chr((ord(a)-step)%256) for a in s])

In [3]:
#Vigener (Видженера)
def encrypt(s, step, alph, map_alph, key):
    j = 0
    r = []
    for a in s:
        r.append(alph[((map_alph[a]+int(key[(j)%len(key)]))%len(alph))])
        j+=1
    return "".join(r)

In [2]:
#Xor
def isNormalSymbol(s):
    a = ord(s)
    if (a >= 48 and a <= 57) or (a >= 65 and a <= 90) or (a >= 97 and a <= 122) or (a == 32):
        return True
    return False
def encrypt(s, key):
    return ("".join(chr(ord(s[i]) ^ ord(key[i%len(key)])) for i in range(0, len(s))))
def decrypt(s):
    len_key = 1
    max_f = 0
    for i in range(1, len(s)//2):
        k = 0
        f = 0
        while(k != len(s)):
            if(s[k] == s[(k+i)%len(s)]):
                f+=1
            k+=1
        if(f > max_f):
            max_f = f
            len_key = i
    m = []
    count_groups = math.ceil(len(s)/len_key)
    for i in range(0, len_key):
        k = i
        mm = set()
        while(k < len(s)):
            ch = chr(ord(s[k])^ord(' '))
            if isNormalSymbol(ch):
                mm.add(ch)
            k+=len_key
        if len(mm) == 0:
            mm.add("NULL")
        m.append(mm)
    print(len_key)
    return m
#abc dga bbc, key = hgd
#(a^h)(b^g)(c^d) (d^h)(g^g)(a^d) (b^h)(b^g)(c^d) (a^g) - original
#(c^d)(a^h)(b^g)(c^d)(d^h)(g^g)(a^d)(b^h)(b^g)
#(b^g)(c^d)(a^h)(b^g)(c^d)(d^h)(g^g)(a^d)(b^h)

#(b^h)(b^g)(c^d)(a^h)(b^g)(c^d)(d^h)(g^g)(a^d)

#(a^h^_)(d^h^_)(b^h^_) (b^g^_)(g^g^_)(b^g^_) (c^d^_)(a^d^_)(c^d^_)

In [12]:
#RSA
def nod(a,b):
    if b == 0:
        return a
    return nod(b, a%b)
p = 53
q = 239 
N = p*q
print(N)
#p*q - p*q/p - p*q/q + 1 = p*q(1-1/p-1/q)+1=p*q(1-(q+p)/p*q)+1=p*q-q-p+1=p(q-1)-(q-1)=(p-1)(q-1), чтд
F = (p-1)*(q-1) #число взаимопростых чисел с N, стоящих до N
MIN = F//5
MIN_N = N//5

c = random.randint(MIN,F)
while nod(F,c) != 1:
    c=random.randint(MIN,F)

d = random.randint(MIN_N,N)
while((d*c)%F != 1):
    d = random.randint(MIN_N,N)
#print(c, d)
#(c, N) - открытый ключ
#(d, N) - закрытый

def encrypt(m,c,N):
    return pow(m,c,N)
def decrypt(e,d,N):
    return pow(e,d,N)


c = 2593
E = [207, 238, 236, 239, 232, 228, 243]
#02685^d%12667 = 00207
#02628^d%12667 = 00238
#05381^d%12667 = 236
#11970^d%12667 = 243
G = []
for e in E:
    g = encrypt(e,c,N)
    G.append(g)
d = 6188+2637
print("d = ", d)
print(G)
for g in G:
    print(decrypt(g,d,N))
#026850262805381...
#dc%F=1 => dc=1+kF
#decrypted = e^d(mod N) = m^cd(mod N) = m^(1+kF)(mod N) = m*m^kF=m*(m^(p-1))^k(q-1) (mod N) = m*1^k(q-1) = m, чтд

12667
d =  8825
[2685, 2628, 5381, 10516, 9694, 8599, 11970]
207
238
236
239
232
228
243


In [179]:
#affine cipher
def encrypt(s, a, b):
    alph = alphs[2]
    m = len(alph)
    return "".join([alph[((a*map_alph[i]+b)%m)] for i in s])
def decrypt(s, a, b,c,d):
    alph = alphs[2]
    m = len(alph)
    #k = F(m)-1
    #a_obr = bin_pow(a, k, m)
    #return "".join([alph[((a_obr*(map_alph[i]+m-b))%m)] for i in s])
    #[ax^3+bx^2+cx+d]=[y]
    res = ""
    for i in s:
        for x in alph:
            if (a*map_alph[x]**3+b*map_alph[x]**2+c*map_alph[x]+d)%m == map_alph[i]:
                res+=x
    return res

In [129]:
#Simple replacing with frequency analyse
def decrypt(s):
    alph = alphs[1]
    d = dict()
    for a in alph:
        d[a]=0
    for ss in s:
        if ss in alph: d[ss]+=1
    f = dict()
    ss = []
    for k in d.keys():
        f[d[k]]=k
        ss.append(d[k])
    ss.sort()
    print(d)